## Imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
import pickle
from scipy.signal import detrend

data_df = pickle.load(open('data/vals_preproc.pkl', 'rb'))
data_df_denoised = pickle.load(open('data/vals_denoised.pkl', 'rb'))
data_df['scans'] = data_df['scans'].apply(lambda x: np.array(x).T)
data_df['scans_0mean'] = data_df['scans_0mean'].apply(lambda x: np.array(x).T)
                                                      
data_df_denoised['scans'] = data_df_denoised['scans'].apply(lambda x: np.array(x).T)
data_df_denoised['scans_0mean'] = data_df_denoised['scans_0mean'].apply(lambda x: np.array(x).T)

In [3]:
from mvmd.mvmd import mvmd
import os
import numpy as np
import scipy.io as sio
from tqdm import tqdm
Ks = range(6, 13)  
alphas = [500, 750, 1000, 1250, 1500]


def unflatten_mvmd_data(u, u_hat, subjects, regions=232):
    assert int(u.shape[1]//regions) == len(subjects), f"The number of subjects does not match the number of rows in u. Got {int(u.shape[1]//regions)} subjects and {len(subjects)} rows."
    data = [
        (
            u[:, i * regions:(i + 1) * regions, :],
            u_hat[:, i * regions:(i + 1) * regions, :]
        )
        for i in range(len(subjects))
    ]
    return dict(zip(subjects, data))

def run_experiment_K(data_df, sample_rate, run_no, Ks, results_folder, alpha = 1000, tol = 1e-7, column = 'scans'):
    filtered_df = data_df[(data_df['sample_rate'] == sample_rate) & (data_df['run'] == run_no)].reset_index(drop=True)
    assert len(filtered_df) == 1, "More than one subject found for the given sample rate and run number."
    for K in tqdm(Ks):
        folder_name = f'{results_folder}/K_{K}'

        os.makedirs(folder_name, exist_ok=True)
        u, u_hat, omega = mvmd(filtered_df[column].values[0], num_modes=K, alpha=alpha, tolerance=tol, freq=filtered_df['sample_rate'].values[0])
        unflattened = unflatten_mvmd_data(u, u_hat, filtered_df['subjects'].values[0])
        for subject, (u_data, u_hat_data) in unflattened.items():
            assert u_data.shape == (K, 232, filtered_df['timepoints'].values[0]), f"u_data shape mismatch for subject {subject}. Expected {filtered_df['timepoints'].values[0]}, got {u_data.shape[0]}"
            sio.savemat(os.path.join(folder_name, f'{subject}_{u_data.shape[-1]}.mat'), {'u': u_data, "u_hat": u_hat_data, "omega": omega})
           
def run_experiment_alpha(data_df, sample_rate, run_no, alphas, results_folder, K = 10, tol = 1e-7, column = 'scans'):
    filtered_df = data_df[(data_df['sample_rate'] == sample_rate) & (data_df['run'] == run_no)].reset_index(drop=True)
    assert len(filtered_df) == 1, "More than one subject found for the given sample rate and run number."
    for alpha in tqdm(alphas):
        folder_name = f'{results_folder}/alpha_{alpha}'

        os.makedirs(folder_name, exist_ok=True)
        u, u_hat, omega = mvmd(filtered_df[column].values[0], num_modes=K, alpha=alpha, tolerance=tol, freq=filtered_df['sample_rate'].values[0])
        unflattened = unflatten_mvmd_data(u, u_hat, filtered_df['subjects'].values[0])
        for subject, (u_data, u_hat_data) in unflattened.items():
            assert u_data.shape == (K, 232, filtered_df['timepoints'].values[0]), f"u_data shape mismatch for subject {subject}. Expected {filtered_df['timepoints'].values[0]}, got {u_data.shape[0]}"
            sio.savemat(os.path.join(folder_name, f'{subject}_{u_data.shape[-1]}.mat'), {'u': u_data, "u_hat": u_hat_data, "omega": omega}) 

def groupruns(data_df, timepoints=375, sr = 0.8, datacolumn = 'scans_0mean'):
    grouped_runs = []
    # Loop through each run
    for run, run_data in data_df.groupby('run'):
        stacked_data = []
        stacked_data_detrended = []
        subjects = []  # List to store subject identifiers for this run
        filenames = []
        stacked_data_detrended_2nd = []
        
        # For each subject in the run, flatten the 232x300 matrix into 232 rows and collect them
        for _, row in run_data.iterrows():
            # Extract the 232x300 matrix for the subject
            scans_matrix = row[datacolumn]
                        
            stacked_data.append(scans_matrix)  # Append the matrix as-is (shape will be (232, 300))
            subjects.append(row['subject'])  # Append the subject to the list
            filenames.append(row['filename'])

        flattened_data = np.vstack(stacked_data)  
        grouped_runs.append([subjects, filenames, sr, timepoints, run, flattened_data])
    return grouped_runs

def run_experiment_K_single(data_df, sample_rate, run_no, Ks, results_folder, alpha = 1000, tol = 1e-7, column = 'scans'):
    filtered_df = data_df[(data_df['sample_rate'] == sample_rate) & (data_df['run'] == run_no)].reset_index(drop=True)
    for K in tqdm(Ks):
        folder_name = f'{results_folder}/K_{K}'

        os.makedirs(folder_name, exist_ok=True)
        for _, row in filtered_df.iterrows():
            subject = row['subject']
            timepoints = row['timepoints']
            u, u_hat, omega = mvmd(row[column], num_modes=K, alpha=alpha, tolerance=tol, freq=filtered_df['sample_rate'].values[0])
            sio.savemat(os.path.join(folder_name, f'{subject}_{timepoints}.mat'), {'u': u, "u_hat": u_hat, "omega": omega})


## Some final signle experiments

In [5]:
data_df_300 = data_df[data_df['sample_rate'] == 2]

for run in range(len(data_df_300.run.unique())):
    run_experiment_K_single(data_df_300, 2.0, run, [6], f'Run1005/Results_0mean/Results_run-{run}_2000ms_/', column='scans_0mean')

100%|██████████| 1/1 [00:10<00:00, 10.59s/it]


In [6]:
data_df_375 = data_df[data_df['timepoints'] == 375] 
data_df_750 = data_df[data_df['timepoints'] == 750] 


for run in range(len(data_df_375.run.unique())):
    run_experiment_K_single(data_df_375, 0.8, run, [10], f'Run1005/Results_0mean/Results_run-{run}_800ms_/', column='scans_0mean')

for run in range(len(data_df_750.run.unique())):
    run_experiment_K_single(data_df_750, 0.8, run, [10], f'Run1005/Results_0mean/Results_run-{run}_800ms_/', column='scans_0mean')

100%|██████████| 1/1 [06:30<00:00, 390.58s/it]


## 200 ms experiments

In [4]:
data_df_300 = data_df[data_df['sample_rate'] == 2]
data_df_300 = data_df_300[data_df_300['timepoints'] == 300]


grouped_runs_300 = groupruns(data_df_300, timepoints = 300, sr = 2.0, datacolumn='scans_0mean')

final_df_300 = pd.DataFrame(grouped_runs_300, columns=['subjects', 'filenames', 'sample_rate', 'timepoints', 'run', 'scans_0mean'])
#final_df_300 = pd.DataFrame(grouped_runs_300, columns=['subjects', 'filenames', 'sample_rate', 'timepoints', 'run', 'scans'])

In [5]:
run_experiment_K(final_df_300, 2.0, 0, [4,5,7,8,9,10], f'Run0605/Results_0mean/Results_run-{0}_2000ms_/', column='scans_0mean')

#for run in range(len(final_df_300)):
#    run_experiment_K(final_df_300, 2.0, run, [6], f'Run0605/Results_0mean/Results_run-{run}_2000ms_/', column='scans_0mean')

100%|██████████| 6/6 [38:21<00:00, 383.55s/it]


## 800 ms experiments

### 1st way, do it seperately for 375 and 750 timepoints

In [6]:
data_df_375 = data_df[data_df['timepoints'] == 375] 
data_df_750 = data_df[data_df['timepoints'] == 750]

grouped_runs_375 = groupruns(data_df_375, 375, datacolumn='scans_0mean')
grouped_runs_750 = groupruns(data_df_750, 750, datacolumn='scans_0mean')

final_df_375 = pd.DataFrame(grouped_runs_375, columns=['subjects', 'filenames', 'sample_rate', 'timepoints', 'run', 'scans_0mean'])
final_df_750 = pd.DataFrame(grouped_runs_750, columns=['subjects', 'filenames', 'sample_rate', 'timepoints', 'run', 'scans_0mean'])

In [7]:
for run in range(len(final_df_375)):
    run_experiment_K(final_df_375, 0.8, run, [10], f'Run0605/Results_0mean/Results_run-{run}_800ms_/', column='scans_0mean')

for run in range(len(final_df_750)):
    run_experiment_K(final_df_750, 0.8, run, [10], f'Run0605/Results_0mean/Results_run-{run}_800ms_/', column='scans_0mean')

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [08:00<00:00, 480.73s/it]


In [8]:
run_experiment_K(final_df_375, 2.0, 0, [9,10,11,12,13], f'Run0605/Results_0mean/Results_run-{0}_800ms_/', column='scans_0mean')
run_experiment_K(final_df_750, 2.0, 0, [9,10,11,12,13], f'Run0605/Results_0mean/Results_run-{0}_800ms_/', column='scans_0mean')

AssertionError: More than one subject found for the given sample rate and run number.

In [39]:
run_experiment_K(final_df_375, 0.8, 1, [10], 'Run0505/Results_detrended/Results_run-0_800ms_/', column='scans_0mean')
run_experiment_K(final_df_375, 0.8, 2, [10], 'Run0505/Results_detrended/Results_run-1_800ms_/', column='scans_0mean')
run_experiment_K(final_df_375, 0.8, 3, [10], 'Run0505/Results_detrended/Results_run-2_800ms_/', column='scans_0mean')
run_experiment_K(final_df_375, 0.8, 4, [10], 'Run0505/Results_detrended/Results_run-3_800ms_/', column='scans_0mean')
run_experiment_K(final_df_375, 0.8, 5, [10], 'Run0505/Results_detrended/Results_run-4_800ms_/', column='scans_0mean')

run_experiment_K(final_df_750, 0.8, 1, [10], 'Run0505/Results_detrended/Results_run-0_800ms_/', column='scans_0mean')
run_experiment_K(final_df_750, 0.8, 2, [10], 'Run0505/Results_detrended/Results_run-1_800ms_/', column='scans_0mean')
run_experiment_K(final_df_750, 0.8, 3, [10], 'Run0505/Results_detrended/Results_run-2_800ms_/', column='scans_0mean')
run_experiment_K(final_df_750, 0.8, 4, [10], 'Run0505/Results_detrended/Results_run-3_800ms_/', column='scans_0mean')
run_experiment_K(final_df_750, 0.8, 5, [10], 'Run0505/Results_detrended/Results_run-4_800ms_/', column='scans_0mean')

100%|██████████| 1/1 [07:15<00:00, 435.44s/it]


In [19]:
run_experiment_alpha(final_df_375, 0.8, 1, [1250, 1500, 2000, 2500, 3000, 5000], 'Run2304/Results_0mean/Results_run-0_800ms_/', column='scans_0mean')
run_experiment_alpha(final_df_750, 0.8, 1, [1250, 1500, 2000, 2500, 3000, 5000], 'Run2304/Results_0mean/Results_run-0_800ms_/', column='scans_0mean')

run_experiment_alpha(final_df_375, 0.8, 2, [1250, 1500, 2000, 2500, 3000, 5000], 'Run2304/Results_0mean/Results_run-1_800ms_/', column='scans_0mean')
run_experiment_alpha(final_df_750, 0.8, 2, [1250, 1500, 2000, 2500, 3000, 5000], 'Run2304/Results_0mean/Results_run-1_800ms_/', column='scans_0mean')

run_experiment_alpha(final_df_375, 0.8, 3, [1250, 1500, 2000, 2500, 3000, 5000], 'Run2304/Results_0mean/Results_run-2_800ms_/', column='scans_0mean')
run_experiment_alpha(final_df_750, 0.8, 3, [1250, 1500, 2000, 2500, 3000, 5000], 'Run2304/Results_0mean/Results_run-2_800ms_/', column='scans_0mean')

run_experiment_alpha(final_df_375, 0.8, 4, [1250, 1500, 2000, 2500, 3000, 5000], 'Run2304/Results_0mean/Results_run-3_800ms_/', column='scans_0mean')
run_experiment_alpha(final_df_750, 0.8, 4, [1250, 1500, 2000, 2500, 3000, 5000], 'Run2304/Results_0mean/Results_run-3_800ms_/', column='scans_0mean')

run_experiment_alpha(final_df_375, 0.8, 5, [1250, 1500, 2000, 2500, 3000, 5000], 'Run2304/Results_0mean/Results_run-4_800ms_/', column='scans_0mean')
run_experiment_alpha(final_df_750, 0.8, 5, [1250, 1500, 2000, 2500, 3000, 5000], 'Run2304/Results_0mean/Results_run-4_800ms_/', column='scans_0mean')

100%|██████████| 6/6 [43:09<00:00, 431.55s/it]


In [8]:
run_experiment_K(final_df_750, 0.8, 1, range(6, 13), 'Run2304/Results_0mean/Results_run-0_800ms_/', column='scans_0mean')
run_experiment_K(final_df_750, 0.8, 2, range(6, 13), 'Run2304/Results_0mean/Results_run-1_800ms_/', column='scans_0mean')
run_experiment_K(final_df_750, 0.8, 3, range(6, 13), 'Run2304/Results_0mean/Results_run-2_800ms_/', column='scans_0mean')
run_experiment_K(final_df_750, 0.8, 4, range(6, 13), 'Run2304/Results_0mean/Results_run-3_800ms_/', column='scans_0mean')
run_experiment_K(final_df_750, 0.8, 5, range(6, 13), 'Run2304/Results_0mean/Results_run-4_800ms_/', column='scans_0mean')

100%|██████████| 7/7 [43:32<00:00, 373.25s/it]


In [9]:
run_experiment_K(final_df_375, 0.8, 5, range(6, 13), 'Run2304/Results_0mean/Results_run-4_800ms_/', column='scans_0mean')

100%|██████████| 7/7 [27:39<00:00, 237.10s/it]


## Saving the results

In [10]:
mvmd_data = pickle.load(open('DATA/mvmd_data.pkl', 'rb'))

In [11]:
mvmd_data.iloc[0]

participant                                               sub-55162
filepath          Run1903/Results_0mean/Results_run-0_800ms_/K_1...
num_timepoints                                                  375
run                                                               0
sample_rate                                                     0.8
u                 [[[-7.684070557813361, -7.609754178828222, -7....
K                                                                10
u_hat             [[[(1.0997298085039044e-10+0j), (2.69045825015...
omegas            [0.011300291801789762, 0.02701443722876842, 0....
Name: 0, dtype: object

In [ ]:

subdirectories = os.listdir(base_folder)
for subdir in tqdm(subdirectories):
    
    

100%|██████████| 10/10 [00:00<00:00, 33797.78it/s]

Run2304/Results_0mean/Results_run-0_2000ms_/K_5
Run2304/Results_0mean/Results_run-0_800ms_/K_10
Run2304/Results_0mean/Results_run-1_2000ms_/K_5
Run2304/Results_0mean/Results_run-1_800ms_/K_10
Run2304/Results_0mean/Results_run-2_2000ms_/K_5
Run2304/Results_0mean/Results_run-2_800ms_/K_10
Run2304/Results_0mean/Results_run-3_800ms_/K_10
Run2304/Results_0mean/Results_run-3_2000ms_/K_5
Run2304/Results_0mean/Results_run-4_800ms_/K_10
Run2304/Results_0mean/Results_run-4_2000ms_/K_5


In [21]:
# Directories to pickle file
subdir_path = ''
long_data = []
base_folder = 'Run2304/Results_0mean'


subdirectories = os.listdir(base_folder)
for subdir in tqdm(subdirectories):
    sr = float(subdir.split('_')[2].split('ms')[0]) / 1000
    subdir_path = ''
    if sr == 0.8:
        subdir_path = os.path.join(base_folder,subdir, 'K_10')
    else:
        subdir_path = os.path.join(base_folder, subdir, 'K_5')


    if os.path.isdir(subdir_path):
        mat_files = [f for f in os.listdir(subdir_path) if f.endswith('.mat')]
        for mat_file in mat_files:
            file_path = os.path.join(subdir_path, mat_file)
            data = loadmat(file_path)
        
            long_data.append({
                'participant': mat_file.split('_')[0],
                'filepath': file_path,
                'num_timepoints': mat_file.split('_')[1].split('.')[0],
                'u': data['u'],
                'u_hat': data['u_hat'],
                'omega': data['omega'],
                'sample_rate': float(subdir.split('_')[2].split('ms')[0]) / 1000,
                'run': int(subdir.split('_')[1].split('-')[1]),
            }
            )


100%|██████████| 10/10 [00:06<00:00,  1.52it/s]


In [26]:
mvmd_data_combined.num_timepoints.unique()

array(['300', '375', '750'], dtype=object)

In [27]:
mvmd_data_combined = pd.DataFrame(long_data)
mvmd_data_combined
pickle.dump(mvmd_data_combined, open('DATA/mvmd_data_concatenated.pkl', 'wb'))